# Ex3 - CreatingDatacube


The notebook shortly demonstrates different ways to create a datacube using `icecube` toolkit. 

In [1]:
import os
import icecube
import glob
from pathlib import Path
from icecube.bin.generate_cube import IceyeProcessGenerateCube

In [2]:
# set paths for the demo
package_abspath = str(Path(icecube.__file__).parent.parent)
resource_dir = os.path.join(package_abspath, "tests/resources")
grd_dir = os.path.join(resource_dir, "grd_stack/")
cube_save_path = os.path.join(package_abspath, "icecube/dataset/temp/my_awesome_cube.nc")
Path(str(Path(cube_save_path).parent)).mkdir(parents=True, exist_ok=True)

vector_labels_fpath = os.path.join(resource_dir, "labels/dummy_vector_labels.json")
masks_labels_fpath = os.path.join(resource_dir, "labels/dummy_mask_labels.json")

**Note:** If you don't have the raster labels saved on your disk, please run the following function and it will generate the labels in the destination folder as specified above. 

In [3]:
from tests.raster_labels_datacube_test import create_run_time_masks_labels
create_run_time_masks_labels()

## icecube CLI

One can use the `icecube` CLI to generate a datacube. Using `icecube --help`, one can see the accepted argument/flags to build the ICEcube. 

Following is the list of positional/optional arguments needed to build the datacube.

- **raster_dir (positional):** path/to/directory where raster are stored 

- **labels-fpath (optional):** path/to/labels.json. The structure of JSON file must be icecube friendly. Please have a look at `icecube.bin.labels_cube.create_json_labels` for more details.

- **cube-save (optional):** path/to/cube.nc where datacube shall be saved

In [4]:
# we can use --help flag to see the accepted args easily
! icecube --help

usage: icecube [-h] [--labels-fpath LABELS_FPATH] [--cube-save CUBE_SAVE]
               raster_dir

CLI support for generating ICEYE datacubes

positional arguments:
  raster_dir            Path/to/directory where raster are stored

optional arguments:
  -h, --help            show this help message and exit
  --labels-fpath LABELS_FPATH
                        path/to/labels.json (in icecube JSON structure) to
                        populate in datacube (Optional)
  --cube-save CUBE_SAVE
                        path/to/cube.nc where datacube shall be saved
                        (Optional)


In [5]:
# Let's build the datacube with only SAR Images
! icecube $grd_dir

09/07/2021 05:56:05 PM - sar_datacube_metadata.py - [INFO] - Building the metadata from the folder /mnt/xor/ICEYE_PACKAGES/icecube/tests/resources/grd_stack/ using GRD
processing rasters for cubes: 100%|██████████████| 3/3 [00:00<00:00, 124.40it/s]
09/07/2021 05:56:05 PM - common_utils.py - [INFO] - create running time is 0.0399 seconds
09/07/2021 05:56:05 PM - generate_cube.py - [INFO] - Skipping labels-cube built, either labels-fpath was not provided or inconsistent extension naming found
09/07/2021 05:56:05 PM - generate_cube.py - [INFO] - Datacube {'Azimuth': 10, 'Band': 3, 'Range': 10} shape built
Generated cube dimensions are: {'Azimuth': 10, 'Band': 3, 'Range': 10}


In [6]:
# building the datacube with SAR data and labels
! icecube $grd_dir --labels-fpath $masks_labels_fpath

09/07/2021 05:56:06 PM - sar_datacube_metadata.py - [INFO] - Building the metadata from the folder /mnt/xor/ICEYE_PACKAGES/icecube/tests/resources/grd_stack/ using GRD
processing rasters for cubes: 100%|██████████████| 3/3 [00:00<00:00, 115.80it/s]
09/07/2021 05:56:06 PM - common_utils.py - [INFO] - create running time is 0.0425 seconds
09/07/2021 05:56:06 PM - sar_datacube_metadata.py - [INFO] - Building the metadata from the folder /mnt/xor/ICEYE_PACKAGES/icecube/tests/resources/grd_stack/ using GRD
/home/iali/anaconda3/envs/icecube_env/lib/python3.8/site-packages/rasterio/__init__.py:207: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
processing rasters for labels cube: 100%|████████| 3/3 [00:00<00:00, 820.91it/s]
09/07/2021 05:56:06 PM - common_utils.py - [INFO] - create running time is 0.0169 seconds
Generated cube dimensions are: {'Azimuth': 10, 'Band': 3, 'R

In [7]:
# building the datacube with SAR data and labels and saving it on the local disk

! icecube $grd_dir --labels-fpath $masks_labels_fpath --cube-save $cube_save_path 
assert(os.path.exists(cube_save_path)) # confirm that file exists.

09/07/2021 05:56:07 PM - sar_datacube_metadata.py - [INFO] - Building the metadata from the folder /mnt/xor/ICEYE_PACKAGES/icecube/tests/resources/grd_stack/ using GRD
processing rasters for cubes: 100%|██████████████| 3/3 [00:00<00:00, 114.91it/s]
09/07/2021 05:56:07 PM - common_utils.py - [INFO] - create running time is 0.0433 seconds
09/07/2021 05:56:07 PM - sar_datacube_metadata.py - [INFO] - Building the metadata from the folder /mnt/xor/ICEYE_PACKAGES/icecube/tests/resources/grd_stack/ using GRD
/home/iali/anaconda3/envs/icecube_env/lib/python3.8/site-packages/rasterio/__init__.py:207: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
processing rasters for labels cube: 100%|████████| 3/3 [00:00<00:00, 920.21it/s]
09/07/2021 05:56:07 PM - common_utils.py - [INFO] - create running time is 0.0167 seconds
Generated cube dimensions are: {'Azimuth': 10, 'Band': 3, 'R

Please note that CLI method reads `icecube/icecube/config.json` file to read the configuration. This file can be modified to change the cube configuration via CLI as needed

## IceyeProcessGenerateCube

Another way to create datacube is to use `icecube.bin.generate_cube.IceyeProcessGenerateCube` class. It provides useful method `create_cube` to generate datacubes. You can find few workflow examples in the script: `icecube.bin.generate_cube` too that will guide you how to create datacubes.

For this demo, we will use the JSON configuration stored in `icecube/tests/resources/json_config/config_use_case5.json`

In [8]:
# Here is a quick look at how the configuration looks like. 

#{
#    "start_date": 20210425,
#    "end_date" : 20210430,
#    "min_incidence_angle" : 20,
#    "max_incidence_angle" : 34,
#    "temporal_resolution" : 1,
#    "temporal_overlap" : 1
#}

# We are asking the cube generator to take into consideration rasters in our stack that span from the 
# dates 25th April to 30th April, and have incidence angles ranging from 20 degrees to 30 degrees. Also the generated 
# cube should 've a and have temporal resolution of 1 day. Temporal overlap denotes that we will accept 
# rasters from the same dates too. 


In [9]:
cube_config_fpath = os.path.join(resource_dir, "json_config/config_use_case5.json")
dc = IceyeProcessGenerateCube.create_cube(
    grd_dir, cube_config_fpath, masks_labels_fpath
)

dc.to_file(cube_save_path)
assert(os.path.exists(cube_save_path)) # confirm that file exists.

09/07/2021 05:56:07 PM - sar_datacube_metadata.py - [INFO] - Building the metadata from the folder /mnt/xor/ICEYE_PACKAGES/icecube/tests/resources/grd_stack/ using GRD
processing rasters for cubes: 100%|██████████| 6/6 [00:00<00:00, 150.53it/s]
09/07/2021 05:56:07 PM - common_utils.py - [INFO] - create running time is 0.0675 seconds
09/07/2021 05:56:07 PM - sar_datacube_metadata.py - [INFO] - Building the metadata from the folder /mnt/xor/ICEYE_PACKAGES/icecube/tests/resources/grd_stack/ using GRD
/home/iali/anaconda3/envs/icecube_env/lib/python3.8/site-packages/rasterio/__init__.py:207: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
processing rasters for labels cube: 100%|██████████| 6/6 [00:00<00:00, 1009.78it/s]
09/07/2021 05:56:07 PM - common_utils.py - [INFO] - create running time is 0.0253 seconds


In [10]:
# we can now see how our datacube looks like:
dc.xrdataset

<xarray.Dataset>
Dimensions:    (Azimuth: 10, Band: 6, Range: 10)
Coordinates:
  * Band       (Band) datetime64[ns] 2021-04-25 2021-04-26 ... 2021-04-30
  * Azimuth    (Azimuth) int64 0 1 2 3 4 5 6 7 8 9
  * Range      (Range) int64 0 1 2 3 4 5 6 7 8 9
Data variables:
    Intensity  (Band, Azimuth, Range) uint16 dask.array<chunksize=(1, 10, 10), meta=np.ndarray>
    Labels     (Band, Azimuth, Range) uint8 dask.array<chunksize=(1, 10, 10), meta=np.ndarray>

Similary feel free to use "vector_labels_fpath" to generate cube as well.

One can also use the following two methods to create the SARdatacube and LabelsDatacube separately. Usually you will not need to interact with these methods unless you are interested in low-level details. 


- `icecube.bin.sar_cube.sar_datacube.SARDatacube.create()` to create SARDatacube
- `icecube.bin.labels_cube.labels_datacube.LabelsDatacube.create()` to create LabelsDatacube    

**Happy Cubing :)**